In [16]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import robotic as ry
import numpy as np
import matplotlib.pyplot as plt
from seqwaynet.keypoint.keypoint_model import NextKeypointPredictor
from seqwaynet.dataset.keypoint_dataset import KeypointDataset
import pathlib
import os
import re
from tqdm import tqdm
import pandas as pd

In [17]:
task = "puck_obs_move"
cur_dir = os.getcwd()
output_dir = f"{cur_dir}/output_contact/{task}"

In [18]:
test_dataset = KeypointDataset(f"/home/kutay/repos/ProgressiveNet/seqwaynet/data/test/keypoint/{task}/sample", stats_file=None)


Loading all data:: 100%|██████████| 360/360 [00:23<00:00, 15.12it/s]


In [19]:
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [20]:
ex_config, ex_cur_contact, ex_cur_gripper, ex_next_keypoint, ex_next_gripper, ex_config_path, ex_contact_path = test_dataset[0]

In [21]:
print(ex_config.shape)

torch.Size([510])


In [22]:
def load_model(path: str):
    model = NextKeypointPredictor(len(ex_config))
    model.load_state_dict(torch.load(path))
    #model.cuda()
    model.eval()
    return model

In [23]:
def replace_keypoint_with_pred(path):
    # Extract the directory and filename parts
    dir_name, file_name = os.path.split(path)
    
    # Use regex to match keypoint_X.csv and replace it with config_X.csv
    new_file_name = re.sub(r'contacts_(\d+)\.csv', r'pred_\1.csv', file_name)
    
    # Join the directory back with the new filename
    new_path = os.path.join(dir_name, new_file_name)
    
    return new_path

In [24]:
# #criterion = nn.HuberLoss(delta=0.5)
# mse = nn.MSELoss()
# bce = nn.BCELoss()

# def evaluate_model(model, test_dataloader):
#     """Evaluate the model on the test set and return the average test loss."""
#     model.eval()  # Set model to evaluation mode
#     test_loss = 0.0

#     with torch.no_grad():  # Disable gradient computation
#         for batch in tqdm(test_dataloader, desc="Evaluating test samples:"):
#             input_configs, input_contacts, input_grippers, next_contacts, next_grippers, config_path, contact_path = batch
            
#             #print(keypoint_path)
#             contact_config_save_path = replace_keypoint_with_pred(contact_path[0])

#             input_configs = input_configs.cuda()
#             input_contacts = input_contacts.cuda()
#             input_grippers = input_configs.cuda()
#             next_contacts = next_contacts.cuda()
#             next_grippers = next_grippers.cuda()

#             # Forward pass
#             contacts, grippers = model.forward(input_configs, input_contacts, input_grippers)
#             print(grippers)

#             outputs = torch.cat((contacts, grippers), dim=1).cpu()
#             # Compute loss
#             mse_loss = mse(contacts, next_contacts)
#             bce_loss = bce(grippers, next_grippers)
#             test_loss += (mse_loss + bce_loss).item()

#             grippers = torch.round(grippers)
#             outputs = outputs.cpu()
#             outputs = pd.DataFrame(outputs).to_csv(contact_config_save_path, header=None, index=None)

#     return test_loss / len(test_dataloader)  # Return average test loss

In [25]:
def evaluate_model(
    model: NextKeypointPredictor,
    val_dataloader: DataLoader,
    criterion_contact,
    criterion_gripper,
):
    """Evaluate the model on the validation set and return the average validation loss."""
    model.eval()  # Set model to evaluation mode
    val_loss_mse = 0.0
    val_loss_bce = 0.0

    with torch.no_grad():  # Disable gradient computation
        for batch in tqdm(val_dataloader, desc="Validation Batches:"):
            (
                input_configs,
                contact_current,
                gripper_idx_current,
                contact_next,
                gripper_idx_next,
                config_path,
                contact_path,
            ) = batch

            # Move data to GPU if available
            input_configs = input_configs.cuda()
            contact_current = contact_current.cuda()
            contact_next = contact_next.cuda()
            gripper_idx_current = gripper_idx_current.view(-1, 1).cuda()
            gripper_idx_next = gripper_idx_next.view(-1, 1).cuda()

            contact_config_save_path = replace_keypoint_with_pred(contact_path[0])
            #print(contact_config_save_path)

            # Forward pass
            contact_pred, gripper_pred = model(
                input_configs, contact_current, gripper_idx_current
            )

            # Compute losses
            loss_contact = criterion_contact(contact_pred, contact_next)
            loss_gripper = criterion_gripper(gripper_pred, gripper_idx_next)
            total_loss = loss_contact + loss_gripper

            val_loss_mse += loss_contact.item()
            val_loss_bce += loss_gripper.item()

            gripper_pred = torch.round(gripper_pred)
            outputs = torch.cat((contact_pred, gripper_pred), dim=1).cpu()
            outputs = pd.DataFrame(outputs).to_csv(contact_config_save_path, header=None, index=None)

    avg_val_loss_mse = val_loss_mse / len(val_dataloader)
    avg_val_loss_bce = val_loss_bce / len(val_dataloader)
    avg_val_loss = avg_val_loss_mse + avg_val_loss_bce
    return (
        avg_val_loss,
        avg_val_loss_mse,
        avg_val_loss_bce,
    )  # Return average validation loss

In [26]:
print(torch.cuda.is_available())

True


In [27]:
model = load_model(f"{output_dir}/best_model.pth")
model.cuda()

NextKeypointPredictor(
  (block1): Sequential(
    (0): Linear(in_features=514, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (block2): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (block3): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (block4): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (output_contact): Linear(in_features=64, out_features=3, bias=True)
  (output_gripper): Sequential(
    (0): Linear

In [28]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

print("Average test loss:", evaluate_model(model, test_dataloader, nn.MSELoss(), nn.BCELoss()))

Validation Batches:: 100%|██████████| 1776/1776 [00:01<00:00, 1201.49it/s]

Average test loss: (0.006288719410375191, 0.006094471291140172, 0.0001942481192350192)


In [29]:
def visualize_episode(model: NextKeypointPredictor, test_dataset: KeypointDataset, ep_idx: int):
    episode_indices = test_dataset.get_episode_indices(ep_idx)

    model.eval()

    with torch.no_grad():
        C = ry.Config()
        f = C.addFrame("cam_frame")
        f.setPose("[-0.4, 0.1, 1, 0.0007963, -1, 0, 0]")
        #f.setAttribute("focalLength", 0.5)  # wide angle
        f.setAttribute("width", 600)  # super wide angle
        f.setAttribute("height", 600)  # super wide angle
        C.view_setCamera(f)

        config_vec, cur_keypoint, next_keypoint_gt, config_path, _ = test_dataset[episode_indices[0]]
        config_path = config_path.replace("csv", "g")
        C.addFile(config_path)

        next_keypoint_ball = C.addFrame("next_keypoint")

        next_keypoint_ball.setShape(ry.ST.sphere, size=[0.02])
        
        next_keypoint_ball.setColor([1.0, 0.0, 0.0])

        next_keypoint_gt_ball = C.addFrame("next_keypoint_gt")
        next_keypoint_gt_ball.setShape(ry.ST.sphere, size=[0.02])
        next_keypoint_gt_ball.setColor([0.0, 1.0, 0.0])
        for idx in episode_indices:
            config_vec, cur_keypoint, next_keypoint_gt, config_path, _ = test_dataset[idx]
            
            #print(config_path)
            
            config_vec = config_vec.view(1, -1).cuda()
            cur_keypoint = cur_keypoint.view(1, -1).cuda()
            #print(config_vec.shape, cur_keypoint.shape)
            next_keypoint = model.forward(config_vec, cur_keypoint)
            next_keypoint = next_keypoint.cpu()

            cur_keypoint = cur_keypoint.view(-1)
            next_keypoint = next_keypoint.view(-1)
            next_keypoint_gt = next_keypoint_gt.view(-1)

            print(next_keypoint, next_keypoint_gt)

            next_keypoint_ball.setPosition(next_keypoint.tolist()[:3])
            next_keypoint_gt_ball.setPosition(next_keypoint_gt.tolist()[:3])
            
            print(cur_keypoint.shape, next_keypoint.shape, next_keypoint_gt.shape)

            #C.addFile(config_path)
            

            C.view(pause=True, message=f"Episode {ep_idx}, step {idx}")
        
        C.view(pause=True)
        


In [30]:
visualize_episode(model, test_dataset, ep_idx=0)

TypeError: expected str, bytes or os.PathLike object, not Tensor